In [1]:
import pandas as pd
from soil_fertility.components.data_pipeline.data_ingestion import DataIngestion
from soil_fertility.components.data_pipeline.data_transformation import DataTransformation
import warnings
warnings.filterwarnings('ignore')


In [2]:
data_path="/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset1.csv"

In [3]:
obj = DataIngestion()
part,train_path,test_path= obj.init_ingestion(
    data_path
)

In [4]:
tranformation_obj = DataTransformation()
part,preprocessor,processed_train_path,processed_test_path = tranformation_obj.transform(
    train_path,test_path,target="Fertility"
)

train_coolumns=  Index(['N', 'P', 'K', 'pH', 'EC', 'OC', 'S', 'Zn', 'Fe', 'Cu', 'Mn', 'B', 'OM',
       'Fertility'],
      dtype='object')
test_coolumns=  Index(['N', 'P', 'K', 'pH', 'EC', 'OC', 'S', 'Zn', 'Fe', 'Cu', 'Mn', 'B', 'OM',
       'Fertility'],
      dtype='object')


In [5]:
train=pd.read_csv(processed_train_path)
test=pd.read_csv(processed_test_path)

In [6]:
from soil_fertility.components.model_pipeline.model_trainer import ModelTrainer

In [7]:
model_trainer_obj = ModelTrainer()

In [8]:
s=model_trainer_obj.init_training(train_data = train, test_data = test, target_column = "Fertility", 
                                 mode='default')

In [9]:
s

{'metrics': {'decision_tree': {'micro_average': {'precision': 0.8693181818181818,
    'recall': 0.8693181818181818,
    'f1_score': 0.8693181818181818},
   'macro_average': {'precision': 0.7003235037949483,
    'recall': 0.7557422969187675,
    'f1_score': 0.7214891436026466},
   'metrics_per_class': {0: {'precision': 0.8421052631578947,
     'recall': 0.9142857142857143,
     'f1_score': 0.8767123287671234,
     'specificity': 0.94},
    1: {'precision': 0.925531914893617,
     'recall': 0.8529411764705882,
     'f1_score': 0.8877551020408163,
     'specificity': 0.8170731707317073},
    2: {'precision': 0.3333333333333333,
     'recall': 0.5,
     'f1_score': 0.4,
     'specificity': 0.9882352941176471}}},
  'random_forest': {'micro_average': {'precision': 0.6022727272727273,
    'recall': 0.6022727272727273,
    'f1_score': 0.6022727272727273},
   'macro_average': {'precision': 0.4451764705882353,
    'recall': 0.44201680672268906,
    'f1_score': 0.4018099547511312},
   'metrics_pe